--- Day 22: Reactor Reboot ---

Operating at these extreme ocean depths has overloaded the submarine's reactor; it needs to be rebooted.

The reactor core is made up of a large 3-dimensional grid made up entirely of cubes, one cube per integer 3-dimensional coordinate (x,y,z). Each cube can be either on or off; at the start of the reboot process, they are all off. (Could it be an old model of a reactor you've seen before?)

To reboot the reactor, you just need to set all of the cubes to either on or off by following a list of reboot steps (your puzzle input). Each step specifies a cuboid (the set of all cubes that have coordinates which fall within ranges for x, y, and z) and whether to turn all of the cubes in that cuboid on or off.

For example, given these reboot steps:

```
on x=10..12,y=10..12,z=10..12
on x=11..13,y=11..13,z=11..13
off x=9..11,y=9..11,z=9..11
on x=10..10,y=10..10,z=10..10
```

The first step (on x=10..12,y=10..12,z=10..12) turns on a 3x3x3 cuboid consisting of 27 cubes:

-    10,10,10
-    10,10,11
-    10,10,12
-    10,11,10
-    10,11,11
-    10,11,12
-    10,12,10
-    10,12,11
-    10,12,12
-    11,10,10
-    11,10,11
-    11,10,12
-    11,11,10
-    11,11,11
-    11,11,12
-    11,12,10
-    11,12,11
-    11,12,12
-    12,10,10
-    12,10,11
-    12,10,12
-    12,11,10
-    12,11,11
-    12,11,12
-    12,12,10
-    12,12,11
-    12,12,12

The second step (on x=11..13,y=11..13,z=11..13) turns on a 3x3x3 cuboid that overlaps with the first. As a result, only 19 additional cubes turn on; the rest are already on from the previous step:

-    11,11,13
-    11,12,13
-    11,13,11
-    11,13,12
-    11,13,13
-    12,11,13
-    12,12,13
-    12,13,11
-    12,13,12
-    12,13,13
-    13,11,11
-    13,11,12
-    13,11,13
-    13,12,11
-    13,12,12
-    13,12,13
-    13,13,11
-    13,13,12
-    13,13,13

The third step (off x=9..11,y=9..11,z=9..11) turns off a 3x3x3 cuboid that overlaps partially with some cubes that are on, ultimately turning off 8 cubes:

-    10,10,10
-    10,10,11
-    10,11,10
-    10,11,11
-    11,10,10
-    11,10,11
-    11,11,10
-    11,11,11

The final step (on x=10..10,y=10..10,z=10..10) turns on a single cube, 10,10,10. After this last step, 39 cubes are on.

The initialization procedure only uses cubes that have x, y, and z positions of at least -50 and at most 50. For now, ignore cubes outside this region.

Here is a larger example:

```
on x=-20..26,y=-36..17,z=-47..7
on x=-20..33,y=-21..23,z=-26..28
on x=-22..28,y=-29..23,z=-38..16
on x=-46..7,y=-6..46,z=-50..-1
on x=-49..1,y=-3..46,z=-24..28
on x=2..47,y=-22..22,z=-23..27
on x=-27..23,y=-28..26,z=-21..29
on x=-39..5,y=-6..47,z=-3..44
on x=-30..21,y=-8..43,z=-13..34
on x=-22..26,y=-27..20,z=-29..19
off x=-48..-32,y=26..41,z=-47..-37
on x=-12..35,y=6..50,z=-50..-2
off x=-48..-32,y=-32..-16,z=-15..-5
on x=-18..26,y=-33..15,z=-7..46
off x=-40..-22,y=-38..-28,z=23..41
on x=-16..35,y=-41..10,z=-47..6
off x=-32..-23,y=11..30,z=-14..3
on x=-49..-5,y=-3..45,z=-29..18
off x=18..30,y=-20..-8,z=-3..13
on x=-41..9,y=-7..43,z=-33..15
on x=-54112..-39298,y=-85059..-49293,z=-27449..7877
on x=967..23432,y=45373..81175,z=27513..53682
```

The last two steps are fully outside the initialization procedure area; all other steps are fully within it. After executing these steps in the initialization procedure region, 590784 cubes are on.

Execute the reboot steps. Afterward, considering only cubes in the region x=-50..50,y=-50..50,z=-50..50, how many cubes are on?


In [ ]:
#load "../Shared/regexp.fsx"
open RegexpMatch

let testInput = """
on x=10..12,y=10..12,z=10..12
on x=11..13,y=11..13,z=11..13
off x=9..11,y=9..11,z=9..11
on x=10..10,y=10..10,z=10..10
"""

let parse (input:string) =
    input.Replace("\r","").Split("\n")
    |> Seq.filter( (<>) "" )
    |> Seq.map (
        function 
        | Regex @"(on|off)\s+x=(-?\d+)..(-?\d+),\s*y=(-?\d+)..(-?\d+),\s*z=(-?\d+)..(-?\d+)" [state; x0; x1; y0; y1; z0; z1] ->
            (int x0, int x1, int y0, int y1, int z0, int z1, state="on")
        | token -> failwithf "Unknown row %s" token
    )

testInput |> parse

index,Item1,Item2,Item3,Item4,Item5,Item6,Item7
0,10,12,10,12,10,12,True
1,11,13,11,13,11,13,True
2,9,11,9,11,9,11,False
3,10,10,10,10,10,10,True


In [ ]:
let drawCuboid (space: Set<int*int*int>) (cuboid: (int*int*int*int*int*int*bool)) =
    let mutable result = space
    let (x0,x1,y0,y1,z0,z1,state) = cuboid
    for z=z0 to z1 do
        for y=y0 to y1 do
            for x=x0 to x1 do
                if state then
                    result <- result.Add( (x,y,z) )
                else
                    result <- result.Remove( (x,y,z) )
    result

drawCuboid Set.empty (0,2,0,2,0,2,true)

index,Item1,Item2,Item3
0,0,0,0
1,0,0,1
2,0,0,2
3,0,1,0
4,0,1,1
5,0,1,2
6,0,2,0
7,0,2,1
8,0,2,2
9,1,0,0


In [ ]:
let drawStep1 (input: seq<int*int*int*int*int*int*bool>) =
    let clip (v:int) = 
        if v < -50 then -50
        elif v > 50 then 50
        else v

    input
    |> Seq.filter( fun (x0,x1,y0,y1,z0,z1,_) -> x0 <= 50 && x1 >= -50 && x0 <= 50 && y1 >= -50 && z0 <= 50 && z1 >= -50 )
    |> Seq.map( fun (x0,x1,y0,y1,z0,z1,state) -> (clip x0, clip x1, clip y0, clip y1, clip z0, clip z1, state) )
    |> Seq.fold drawCuboid Set.empty
    |> Seq.length

testInput |> parse |> drawStep1  // 39

39

In [ ]:
let testInput2="""
on x=-20..26,y=-36..17,z=-47..7
on x=-20..33,y=-21..23,z=-26..28
on x=-22..28,y=-29..23,z=-38..16
on x=-46..7,y=-6..46,z=-50..-1
on x=-49..1,y=-3..46,z=-24..28
on x=2..47,y=-22..22,z=-23..27
on x=-27..23,y=-28..26,z=-21..29
on x=-39..5,y=-6..47,z=-3..44
on x=-30..21,y=-8..43,z=-13..34
on x=-22..26,y=-27..20,z=-29..19
off x=-48..-32,y=26..41,z=-47..-37
on x=-12..35,y=6..50,z=-50..-2
off x=-48..-32,y=-32..-16,z=-15..-5
on x=-18..26,y=-33..15,z=-7..46
off x=-40..-22,y=-38..-28,z=23..41
on x=-16..35,y=-41..10,z=-47..6
off x=-32..-23,y=11..30,z=-14..3
on x=-49..-5,y=-3..45,z=-29..18
off x=18..30,y=-20..-8,z=-3..13
on x=-41..9,y=-7..43,z=-33..15
on x=-54112..-39298,y=-85059..-49293,z=-27449..7877
on x=967..23432,y=45373..81175,z=27513..53682
"""

testInput2 |> parse |> drawStep1  // 590784

590784

In [ ]:
let task = System.IO.File.ReadAllText "task.txt"
task |> parse |> drawStep1  // 642125

642125

--- Part Two ---

Now that the initialization procedure is complete, you can reboot the reactor.

Starting with all cubes off, run all of the reboot steps for all cubes in the reactor.

Consider the following reboot steps:

```
on x=-5..47,y=-31..22,z=-19..33
on x=-44..5,y=-27..21,z=-14..35
on x=-49..-1,y=-11..42,z=-10..38
on x=-20..34,y=-40..6,z=-44..1
off x=26..39,y=40..50,z=-2..11
on x=-41..5,y=-41..6,z=-36..8
off x=-43..-33,y=-45..-28,z=7..25
on x=-33..15,y=-32..19,z=-34..11
off x=35..47,y=-46..-34,z=-11..5
on x=-14..36,y=-6..44,z=-16..29
on x=-57795..-6158,y=29564..72030,z=20435..90618
on x=36731..105352,y=-21140..28532,z=16094..90401
on x=30999..107136,y=-53464..15513,z=8553..71215
on x=13528..83982,y=-99403..-27377,z=-24141..23996
on x=-72682..-12347,y=18159..111354,z=7391..80950
on x=-1060..80757,y=-65301..-20884,z=-103788..-16709
on x=-83015..-9461,y=-72160..-8347,z=-81239..-26856
on x=-52752..22273,y=-49450..9096,z=54442..119054
on x=-29982..40483,y=-108474..-28371,z=-24328..38471
on x=-4958..62750,y=40422..118853,z=-7672..65583
on x=55694..108686,y=-43367..46958,z=-26781..48729
on x=-98497..-18186,y=-63569..3412,z=1232..88485
on x=-726..56291,y=-62629..13224,z=18033..85226
on x=-110886..-34664,y=-81338..-8658,z=8914..63723
on x=-55829..24974,y=-16897..54165,z=-121762..-28058
on x=-65152..-11147,y=22489..91432,z=-58782..1780
on x=-120100..-32970,y=-46592..27473,z=-11695..61039
on x=-18631..37533,y=-124565..-50804,z=-35667..28308
on x=-57817..18248,y=49321..117703,z=5745..55881
on x=14781..98692,y=-1341..70827,z=15753..70151
on x=-34419..55919,y=-19626..40991,z=39015..114138
on x=-60785..11593,y=-56135..2999,z=-95368..-26915
on x=-32178..58085,y=17647..101866,z=-91405..-8878
on x=-53655..12091,y=50097..105568,z=-75335..-4862
on x=-111166..-40997,y=-71714..2688,z=5609..50954
on x=-16602..70118,y=-98693..-44401,z=5197..76897
on x=16383..101554,y=4615..83635,z=-44907..18747
off x=-95822..-15171,y=-19987..48940,z=10804..104439
on x=-89813..-14614,y=16069..88491,z=-3297..45228
on x=41075..99376,y=-20427..49978,z=-52012..13762
on x=-21330..50085,y=-17944..62733,z=-112280..-30197
on x=-16478..35915,y=36008..118594,z=-7885..47086
off x=-98156..-27851,y=-49952..43171,z=-99005..-8456
off x=2032..69770,y=-71013..4824,z=7471..94418
on x=43670..120875,y=-42068..12382,z=-24787..38892
off x=37514..111226,y=-45862..25743,z=-16714..54663
off x=25699..97951,y=-30668..59918,z=-15349..69697
off x=-44271..17935,y=-9516..60759,z=49131..112598
on x=-61695..-5813,y=40978..94975,z=8655..80240
off x=-101086..-9439,y=-7088..67543,z=33935..83858
off x=18020..114017,y=-48931..32606,z=21474..89843
off x=-77139..10506,y=-89994..-18797,z=-80..59318
off x=8476..79288,y=-75520..11602,z=-96624..-24783
on x=-47488..-1262,y=24338..100707,z=16292..72967
off x=-84341..13987,y=2429..92914,z=-90671..-1318
off x=-37810..49457,y=-71013..-7894,z=-105357..-13188
off x=-27365..46395,y=31009..98017,z=15428..76570
off x=-70369..-16548,y=22648..78696,z=-1892..86821
on x=-53470..21291,y=-120233..-33476,z=-44150..38147
off x=-93533..-4276,y=-16170..68771,z=-104985..-24507
```

After running the above reboot steps, 2758514936282235 cubes are on. (Just for fun, 474140 of those are also in the initialization procedure region.)

Starting again with all cubes off, execute all reboot steps. Afterward, considering all cubes, how many cubes are on?


In [ ]:
// A - B
let subtractCuboids (left: (int*int) array ) (right:(int*int) array) =
    let rec splitCoord (coord:int) (parts: seq<((int*int) array) * bool>)=
        let (l0, l1) = left[coord]
        let (r0, r1) = right[coord]
        // printfn "%i %i     %i %i" l0 l1 r0 r1
        if l0 > r1 || l1 < r0 then
            parts 
            |> Seq.map ( fun (partCoords, _) -> (partCoords, false))
        else 
            let intervals = 
                // R completely covering
                if r0 <= l0 && r1 >= l1 then
                    [(l0, l1, true)]
                // R completely inside
                elif r0 >l0 && r1 < l1 then
                    [(l0,r0-1, false); (r0, r1, true); (r1+1,l1,false)]
                // R under and over
                elif r0 <= l0 then
                    [(l0,r1,true); (r1+1,l1,false)]
                // R over and above
                else 
                    [(l0,r0-1,false); (r0,l1,true)]
            parts 
            |> Seq.map ( fun (partCoords, partInside) ->
                intervals
                |> Seq.map ( fun (x0, x1, inside) ->
                    ( 
                        [|
                            if coord = 0 then (x0, x1) else partCoords[0]
                            if coord = 1 then (x0, x1) else partCoords[1]
                            if coord = 2 then (x0, x1) else partCoords[2]
                        |]
                        ,
                        partInside && inside
                    )
                )
            )
            |> Seq.concat

    let newCubes =
        (seq{ ( left, true ) })
        |> splitCoord 0
        |> splitCoord 1
        |> splitCoord 2
        |> Seq.cache

    if (Seq.tryFind ( fun (coords, inside) -> inside ) newCubes ).IsNone then
        seq { left }

    else 
        newCubes
        |> Seq.filter ( fun (coords, inside) -> not inside )
        |> Seq.map fst

// subtractCuboids [|(0,5); (0,5); (0,5)|]  [|(2,10); (2,10); (2,10)|]   // right over and above
// subtractCuboids [|(0,5); (0,5); (0,5)|]  [|(-2,10); (2,10); (2,10)|]  // right covering
// subtractCuboids [|(0,5); (0,5); (0,5)|]  [|(2,3); (2,10); (2,10)|]  // right within
// subtractCuboids [|(0,5); (0,5); (0,5)|]  [|(-2,3); (2,10); (2,10)|]  // right under and over
// subtractCuboids [|(0,5); (0,5); (0,5)|]  [|(-2,10); (-2,10); (-2,10)|]   // right covering
// subtractCuboids [|(0,5); (0,5); (0,5)|]  [|(2,10); (2,10); (8,10)|]   // outside
//subtractCuboids [|(10,12); (10,12); (0,5)|]  [|(2,10); (2,10); (8,10)|]   // outside
// subtractCuboids [|(10,12); (10,12); (10,12)|]  [|(11,13); (11,13); (11,13)|] |> mergeCuboids // test 1
subtractCuboids [|(11,13); (11,13); (11,13)|] [|(10,12); (10,12); (10,12)|]  // test 2
        

index,value
0,"[ ( 11, 12 ), ( 11, 12 ), ( 13, 13 ) ]"
1,"[ ( 11, 12 ), ( 13, 13 ), ( 11, 12 ) ]"
2,"[ ( 11, 12 ), ( 13, 13 ), ( 13, 13 ) ]"
3,"[ ( 13, 13 ), ( 11, 12 ), ( 11, 12 ) ]"
4,"[ ( 13, 13 ), ( 11, 12 ), ( 13, 13 ) ]"
5,"[ ( 13, 13 ), ( 13, 13 ), ( 11, 12 ) ]"
6,"[ ( 13, 13 ), ( 13, 13 ), ( 13, 13 ) ]"


In [ ]:
let mergeCuboids (input: seq<(int*int) array> ) =
    let rec merge (left: (int*int) array) (right: (int*int) array ) =
        let coords = seq { 0..2 }
        let notMatchingCoords = coords |> Seq.filter( fun i -> left[i]<>right[i] )
        if notMatchingCoords |> Seq.length = 1 then
            let coord = notMatchingCoords |> Seq.head
            let (l0,l1) = left[coord]
            let (r0,r1) = right[coord]
            let newCoord = 
                if l1+1=r0 then 
                    Some (l0, r1)
                elif r1+1=l0 then
                    Some (r0,l1)
                else
                    None
            if newCoord.IsSome then
                Some ( left |> Array.mapi ( fun i value -> if i = coord then newCoord.Value else value ) )
            else 
                None
        else
            None

    let rec mergeOne (merged: List<(int*int) array>) (todo: list<(int*int) array>) modified =
        match todo with
        | one::rest ->
            let mutable found = false
            let newMerged = 
                merged
                |> List.map ( fun cube ->
                    if found then 
                        cube
                    else
                        let result = merge cube one
                        if result.IsSome then
                            found <- true
                            result.Value
                        else
                            cube

                )
            if found then
                mergeOne newMerged rest true
            else
                mergeOne (one::merged) rest modified
        | [] when modified -> mergeOne [] merged false
        | [] -> merged

    mergeOne [] (input |> List.ofSeq ) false

// subtractCuboids [|(0,5); (0,5); (0,5)|]  [|(2,10); (2,10); (2,10)|]  |> mergeCuboids  // right over and above
subtractCuboids [|(0,10); (0,10); (0,10)|]  [|(2,7); (2,7); (2,7)|]  |> mergeCuboids  // bit off center

index,value
0,"[ ( 8, 10 ), ( 0, 10 ), ( 0, 10 ) ]"
1,"[ ( 2, 7 ), ( 8, 10 ), ( 2, 10 ) ]"
2,"[ ( 2, 7 ), ( 2, 7 ), ( 8, 10 ) ]"
3,"[ ( 2, 7 ), ( 2, 10 ), ( 0, 1 ) ]"
4,"[ ( 2, 7 ), ( 0, 1 ), ( 0, 10 ) ]"
5,"[ ( 0, 1 ), ( 0, 10 ), ( 0, 10 ) ]"


In [ ]:
let noOverlap (left: (int*int) array) (right: (int*int) array) =
    Seq.exists2 ( fun l r -> 
            let (l0, l1) = l
            let (r0, r1) = r
            r0 > l1 || r1 < l0
        ) 
        left 
        right

noOverlap [|(0,5); (0,5); (0,5)|]  [|(6,10); (2,10); (2,10)|]

True

In [ ]:
let addCuboidToList (state: seq<(int*int) array> ) (cuboid: (int*int) array ) =
    state
    |> Seq.fold ( fun fragments existingCuboid -> 
            fragments
            |> List.map( fun fragment ->
                if noOverlap fragment existingCuboid then
                    [ fragment ]
                else
                    subtractCuboids fragment existingCuboid |> mergeCuboids
            )
            |> List.concat
        )
        [cuboid]
    |> Seq.append state
    |> mergeCuboids

addCuboidToList
    (seq {
        [|(1,4); (1,4); (1,4)|]
        [|(0,1); (0,1); (0,1)|]
    })
    [|(0,2); (0,2); (0,2)|]  


index,value
0,"[ ( 0, 0 ), ( 2, 2 ), ( 0, 2 ) ]"
1,"[ ( 0, 0 ), ( 0, 1 ), ( 2, 2 ) ]"
2,"[ ( 2, 2 ), ( 0, 0 ), ( 0, 2 ) ]"
3,"[ ( 1, 1 ), ( 0, 0 ), ( 2, 2 ) ]"
4,"[ ( 2, 2 ), ( 1, 2 ), ( 0, 0 ) ]"
5,"[ ( 1, 1 ), ( 2, 2 ), ( 0, 0 ) ]"
6,"[ ( 0, 1 ), ( 0, 1 ), ( 0, 1 ) ]"
7,"[ ( 1, 4 ), ( 1, 4 ), ( 1, 4 ) ]"


In [ ]:
let subtractCuboidFromList (state: seq<(int*int) array> ) (cuboid: (int*int) array ) =
    state
    |> Seq.map ( fun existingCuboid -> 
            if noOverlap existingCuboid cuboid then
                [ existingCuboid ]
            else
                subtractCuboids existingCuboid cuboid |> mergeCuboids
        )
    |> Seq.concat
    |> mergeCuboids

subtractCuboidFromList
    (seq {
        [|(0,2); (0,2); (0,2)|]  
    })
    [|(1,4); (1,4); (1,4)|]

index,value
0,"[ ( 0, 0 ), ( 0, 2 ), ( 0, 2 ) ]"
1,"[ ( 1, 2 ), ( 0, 0 ), ( 0, 2 ) ]"
2,"[ ( 1, 2 ), ( 1, 2 ), ( 0, 0 ) ]"


In [ ]:
let drawStep2 clipped (input: seq<int*int*int*int*int*int*bool>) =
    let clip (v:int) = 
        if v < -50 then -50
        elif v > 50 then 50
        else v

    let processed = 
        if clipped then
            input
            |> Seq.filter( fun (x0,x1,y0,y1,z0,z1,_) -> x0 <= 50 && x1 >= -50 && x0 <= 50 && y1 >= -50 && z0 <= 50 && z1 >= -50 )
            |> Seq.map( fun (x0,x1,y0,y1,z0,z1,state) -> (clip x0, clip x1, clip y0, clip y1, clip z0, clip z1, state) )
        else
            input

    processed
    |> Seq.fold ( fun (cubes:seq<(int*int)array>) newCuboid ->
        let (x0,x1,y0,y1,z0,z1,add) = newCuboid
        if add then
            addCuboidToList cubes [|(x0,x1); (y0,y1); (z0,z1)|]
        else 
            subtractCuboidFromList cubes [|(x0,x1); (y0,y1); (z0,z1)|]
        )
        []
    |> Seq.sumBy (
        Array.map (fun (x0,x1) -> int64(x1 - x0 + 1) )
        >> Array.reduce (*) 
    )

testInput |> parse |> drawStep2 true 

39

In [ ]:
testInput2 |> parse |> drawStep2 true // 590784

590784

In [ ]:
let testInput3 = """
on x=-5..47,y=-31..22,z=-19..33
on x=-44..5,y=-27..21,z=-14..35
on x=-49..-1,y=-11..42,z=-10..38
on x=-20..34,y=-40..6,z=-44..1
off x=26..39,y=40..50,z=-2..11
on x=-41..5,y=-41..6,z=-36..8
off x=-43..-33,y=-45..-28,z=7..25
on x=-33..15,y=-32..19,z=-34..11
off x=35..47,y=-46..-34,z=-11..5
on x=-14..36,y=-6..44,z=-16..29
on x=-57795..-6158,y=29564..72030,z=20435..90618
on x=36731..105352,y=-21140..28532,z=16094..90401
on x=30999..107136,y=-53464..15513,z=8553..71215
on x=13528..83982,y=-99403..-27377,z=-24141..23996
on x=-72682..-12347,y=18159..111354,z=7391..80950
on x=-1060..80757,y=-65301..-20884,z=-103788..-16709
on x=-83015..-9461,y=-72160..-8347,z=-81239..-26856
on x=-52752..22273,y=-49450..9096,z=54442..119054
on x=-29982..40483,y=-108474..-28371,z=-24328..38471
on x=-4958..62750,y=40422..118853,z=-7672..65583
on x=55694..108686,y=-43367..46958,z=-26781..48729
on x=-98497..-18186,y=-63569..3412,z=1232..88485
on x=-726..56291,y=-62629..13224,z=18033..85226
on x=-110886..-34664,y=-81338..-8658,z=8914..63723
on x=-55829..24974,y=-16897..54165,z=-121762..-28058
on x=-65152..-11147,y=22489..91432,z=-58782..1780
on x=-120100..-32970,y=-46592..27473,z=-11695..61039
on x=-18631..37533,y=-124565..-50804,z=-35667..28308
on x=-57817..18248,y=49321..117703,z=5745..55881
on x=14781..98692,y=-1341..70827,z=15753..70151
on x=-34419..55919,y=-19626..40991,z=39015..114138
on x=-60785..11593,y=-56135..2999,z=-95368..-26915
on x=-32178..58085,y=17647..101866,z=-91405..-8878
on x=-53655..12091,y=50097..105568,z=-75335..-4862
on x=-111166..-40997,y=-71714..2688,z=5609..50954
on x=-16602..70118,y=-98693..-44401,z=5197..76897
on x=16383..101554,y=4615..83635,z=-44907..18747
off x=-95822..-15171,y=-19987..48940,z=10804..104439
on x=-89813..-14614,y=16069..88491,z=-3297..45228
on x=41075..99376,y=-20427..49978,z=-52012..13762
on x=-21330..50085,y=-17944..62733,z=-112280..-30197
on x=-16478..35915,y=36008..118594,z=-7885..47086
off x=-98156..-27851,y=-49952..43171,z=-99005..-8456
off x=2032..69770,y=-71013..4824,z=7471..94418
on x=43670..120875,y=-42068..12382,z=-24787..38892
off x=37514..111226,y=-45862..25743,z=-16714..54663
off x=25699..97951,y=-30668..59918,z=-15349..69697
off x=-44271..17935,y=-9516..60759,z=49131..112598
on x=-61695..-5813,y=40978..94975,z=8655..80240
off x=-101086..-9439,y=-7088..67543,z=33935..83858
off x=18020..114017,y=-48931..32606,z=21474..89843
off x=-77139..10506,y=-89994..-18797,z=-80..59318
off x=8476..79288,y=-75520..11602,z=-96624..-24783
on x=-47488..-1262,y=24338..100707,z=16292..72967
off x=-84341..13987,y=2429..92914,z=-90671..-1318
off x=-37810..49457,y=-71013..-7894,z=-105357..-13188
off x=-27365..46395,y=31009..98017,z=15428..76570
off x=-70369..-16548,y=22648..78696,z=-1892..86821
on x=-53470..21291,y=-120233..-33476,z=-44150..38147
off x=-93533..-4276,y=-16170..68771,z=-104985..-24507
"""
testInput3 |> parse |> drawStep2 false  // 2758514936282235

2758514936282235

In [ ]:
task |> parse |> drawStep2 false  // 1235164413198198

1235164413198198

In [ ]:
let drawStep2fast clipped (input: seq<int*int*int*int*int*int*bool>) =
    let clip (v:int) = 
        if v < -50 then -50
        elif v > 50 then 50
        else v

    let processed = 
        if clipped then
            input
            |> Seq.filter( fun (x0,x1,y0,y1,z0,z1,_) -> x0 <= 50 && x1 >= -50 && x0 <= 50 && y1 >= -50 && z0 <= 50 && z1 >= -50 )
            |> Seq.map( fun (x0,x1,y0,y1,z0,z1,state) -> (clip x0, clip x1, clip y0, clip y1, clip z0, clip z1, state) )
        else
            input

    processed
    |> Seq.cache
    |> Seq.fold ( fun (cubes:seq<int * int * int * int * int * int * int>) newCuboid ->
            let (x0,x1,y0,y1,z0,z1,add) = newCuboid
            cubes 
            |> Seq.filter ( fun (lx0, lx1, ly0, ly1, lz0, lz1, lsign ) -> lx0 <= x1 && lx1 >= x0 && ly0 <= y1 && ly1 >= y0 && lz0 <= z1 && lz1 >= z0  )
            |> Seq.map ( fun (lx0, lx1, ly0, ly1, lz0, lz1, lsign ) ->
                (
                    Math.Max(x0, lx0),
                    Math.Min(x1, lx1),
                    Math.Max(y0, ly0),
                    Math.Min(y1, ly1),
                    Math.Max(z0, lz0),
                    Math.Min(z1, lz1),
                    -lsign
                )
            )
            |> Seq.append cubes
            |> Seq.append ( if add then [ (x0, x1, y0, y1, z0, z1, 1)] else [] )
            |> Seq.cache
        )
        []
    |> Seq.map ( fun (lx0, lx1, ly0, ly1, lz0, lz1, lsign ) -> int64( int64(lx1-lx0+1)*int64(ly1-ly0+1)*int64(lz1-lz0+1)*int64(lsign) ) )
    |> Seq.sum

testInput |> parse |> drawStep2fast true 

39

In [ ]:
testInput2 |> parse |> drawStep2fast true // 590784

590784

In [ ]:
testInput3 |> parse |> drawStep2fast false  // 2758514936282235

2758514936282235

In [ ]:
task |> parse |> drawStep2fast false  // 1235164413198198

1235164413198198